In [ ]:


import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# ##### Merge 12 months of sales data into a single csv file 

In [ ]:


files =[file for file in os.listdir("D:\projects\sales\Sales_Data")]
for file in files:
    print(file)

In [ ]:


path = "D:\projects\sales\Sales_Data"

#blank dataframe
all_data = pd.DataFrame()

for file in files:
    current_df = pd.read_csv(path+"/"+file)
    all_data = pd.concat([all_data, current_df])
    
all_data.shape


# ##### convert it into dataset

In [ ]:


all_data.to_csv('D:\projects\sales\Sales_Data/all_data.csv',index=False)


# ##### Data cleaning and formatting

In [ ]:


all_data.dtypes

In [ ]:


all_data.head()

In [ ]:


all_data.isnull().sum()

In [ ]:


all_data = all_data.dropna(how='all')
all_data.shape


# ##### What is the best month for sale?

In [ ]:


'04/19/19 08:46'.split('/')[0]

In [ ]:


def month(x):
    return x.split('/')[0]


# ##### add month col

In [ ]:


all_data['Month']=all_data['Order Date'].apply(month)

In [ ]:


all_data.dtypes

In [ ]:


all_data['Month']=all_data['Month'].astype(int)

In [ ]:


all_data['Month'].unique()

In [ ]:


filter=all_data['Month']=='Order Date'
len(all_data[~filter])

In [ ]:


all_data=all_data[~filter]

In [ ]:


all_data.shape

In [ ]:


all_data.head()

In [ ]:


all_data['Month']=all_data['Month'].astype(int)

In [ ]:


all_data.dtypes

In [ ]:


all_data['Price Each']=all_data['Price Each'].astype(float)

In [ ]:


all_data['Quantity Ordered']=all_data['Quantity Ordered'].astype(int)

In [ ]:


all_data['sales']=all_data['Quantity Ordered']*all_data['Price Each']
all_data.head(5)

In [ ]:


all_data.groupby('Month')['sales'].sum()

In [ ]:


months=range(1,13)
plt.bar(months,all_data.groupby('Month')['sales'].sum())
plt.xticks(months)
plt.ylabel('Sales in USD ($)')
plt.xlabel('Month number')
plt.show()


# ##### Which city has max order

In [ ]:


'917 1st St, Dallas, TX 75001'.split(',')[1]

In [ ]:


def city(x):
    return x.split(',')[1]

In [ ]:


all_data['city']=all_data['Purchase Address'].apply(city)

In [ ]:


all_data.groupby('city')['city'].count()

In [ ]:


plt.bar(all_data.groupby('city')['city'].count().index,all_data.groupby('city')['city'].count())
plt.xticks(rotation='vertical')
plt.ylabel('received orders')
plt.xlabel('city names')
plt.show()

In [ ]:





# ##### What time should we display advertisements to maximise for product purchase?

In [ ]:


all_data['Order Date'][0].dtype

In [ ]:


all_data['Hour'] = pd.to_datetime(all_data['Order Date']).dt.hour

In [ ]:


keys=[]
hour=[]
for key,hour_df in all_data.groupby('Hour'):
    keys.append(key)
    hour.append(len(hour_df))

In [ ]:


plt.grid()
plt.plot(keys,hour)


# ##### between 12pm and 7pm is probably the best time to advertise to maximise product purchase

In [ ]:





# #### What product sold the most? &  Why?

In [ ]:


all_data.groupby('Product')['Quantity Ordered'].sum().plot(kind='bar')

In [ ]:


all_data.groupby('Product')['Price Each'].mean()

In [ ]:


products=all_data.groupby('Product')['Quantity Ordered'].sum().index
quantity=all_data.groupby('Product')['Quantity Ordered'].sum()
prices=all_data.groupby('Product')['Price Each'].mean()

In [ ]:


plt.figure(figsize=(40,24))
fig,ax1 = plt.subplots()
ax2=ax1.twinx()
ax1.bar(products, quantity, color='g')
ax2.plot(products, prices, 'b-')
ax1.set_xticklabels(products, rotation='vertical', size=8)


# ##### The top selling product is 'AAA Batteries'. The top selling products seem to have a correlation with the price of the product. The cheaper the product higher the quantity ordered and vice versa.

In [ ]:


all_data.shape


# ##### What products are most often sold together?

# #### note: keep orders that have same order Id,are sold mostly together

In [ ]:


df=all_data[all_data['Order ID'].duplicated(keep=False)]
df.head(20)

In [ ]:


#create grouped col 
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

In [ ]:


df.head()

In [ ]:


df.shape

In [ ]:


#lets drop out all duplicate Order ID
df2 = df.drop_duplicates(subset=['Order ID'])

In [ ]:


df2['Grouped'].value_counts()[0:5].plot.pie()

In [ ]:


import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:


values=df2['Grouped'].value_counts()[0:5]
labels=df['Grouped'].value_counts()[0:5].index

In [ ]:


trace=go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=25),
              pull=[0, 0, 0,0.2, 0]
               )

In [ ]:


iplot([trace])